In [1]:
import os 
os.environ['MLFLOW_TRACKING_URI']='https://dagshub.com/ankitbiswas008/datascience_deployment.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME']='ankitbiswas008'
os.environ['MLFLOW_TRACKING_PASSWORD']='eb9fc481646c52c5e2740778b48ba9d38d70f9a5'

In [2]:
import os
%pwd


'c:\\MLOps\\datascience_deployment_firstproject\\datascience_deployment\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\MLOps\\datascience_deployment_firstproject\\datascience_deployment'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path 
    model_path: Path 
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri:str 
    

In [7]:
from src.datascience_deployment.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH,SCHEMA_FILE_PATH
from src.datascience_deployment.utils.common import read_yaml,create_directories,save_json

In [16]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self)->ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema = self.schema.TARGET
        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params= params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri='https://dagshub.com/ankitbiswas008/datascience_deployment.mlflow'
            
        )

        return model_evaluation_config



In [17]:
import pandas as pd
import os 
from src.datascience_deployment import logger
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [27]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config=config

    def evaluation_metrics(self,actual,pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)
        return rmse, mae, r2 
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column],axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme 
        print(tracking_url_type_store)

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            (rmse, mae, r2) = self.evaluation_metrics(test_y,predicted_qualities)

            # Saving metrics in local 
            scores = {"rmse":rmse,"mae":mae,"r2":r2}
            # save_json(path=Path(self.config.metric_file_name),data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("r2",r2)
            mlflow.log_metric("mae",mae)

            #Model registry does not work with file store
            if tracking_url_type_store!="file":
                print(model)
                mlflow.sklearn.log_model(model,"model", registered_model_name = "ElasticModel")

            else:
                mlflow.sklearn.log_model(model,"model")


In [28]:
try:
    config=ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e 

[2025-03-24 14:36:23,727]:INFO:common:yaml file: config\config.yaml loaded successfully:]
[2025-03-24 14:36:23,731]:INFO:common:yaml file: params.yaml loaded successfully:]
[2025-03-24 14:36:23,735]:INFO:common:yaml file: schema.yaml loaded successfully:]
[2025-03-24 14:36:23,736]:INFO:common:created directory at: artifacts:]
[2025-03-24 14:36:23,737]:INFO:common:created directory at: artifacts/model_evaluation:]
https
ElasticNet(alpha=0.2, l1_ratio=0.1, random_state=42)


2025/03/24 14:36:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticModel'.
2025/03/24 14:36:44 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticModel, version 1
Created version '1' of model 'ElasticModel'.


🏃 View run flawless-pig-233 at: https://dagshub.com/ankitbiswas008/datascience_deployment.mlflow/#/experiments/0/runs/4694c5d19eeb4ed4b5837946d476e011
🧪 View experiment at: https://dagshub.com/ankitbiswas008/datascience_deployment.mlflow/#/experiments/0
